<a href="https://colab.research.google.com/github/chaitraDev/GenAI-for-marketing/blob/main/Cognizant_RFM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import datetime

# Data Import

In [ ]:
def import_data():
  customers = pd.read_csv("/content/drive/MyDrive/Cognizant/CSVs/customers.csv")
  transactions = pd.read_csv("/content/drive/MyDrive/Cognizant/CSVs/transactions.csv")
  transactions["TransactionDate"] = pd.to_datetime(transactions.TransactionDate).dt.date
  return customers, transactions

# EDA

In [ ]:
# to take the imported data
customers,transactions = import_data()

<ipython-input-7-66618876eaa0>:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  transactions["TransactionDate"] = pd.to_datetime(transactions.TransactionDate).dt.date


In [ ]:
customers.shape

(539676, 5)

In [ ]:
customers.head()

CustomerID CustomerDOB CustGender CustLocation  CustAccountBalance
0   C5841053     10/1/94          F   JAMSHEDPUR            17819.05
1   C2142763      4/4/57          M      JHAJJAR             2270.69
2   C4417068    26/11/96          F       MUMBAI            17874.44
3   C5342380     14/9/73          F       MUMBAI           866503.21
4   C9031234     24/3/88          F  NAVI MUMBAI             6714.43

In [ ]:
transactions.head()

TransactionID CustomerID TransactionDate  TransactionTime  \
0            T1   C5841053      2016-02-08           143207   
1            T2   C2142763      2016-02-08           141858   
2            T3   C4417068      2016-02-08           142712   
3            T4   C5342380      2016-02-08           142714   
4            T5   C9031234      2016-02-08           181156   

   TransactionAmount (INR)  
0                     25.0  
1                  27999.0  
2                    459.0  
3                   2060.0  
4                   1762.5

# Quantifying Customer Relation

## Preparing the RFM columns

In [ ]:
transactions.dtypes

TransactionID               object
CustomerID                  object
TransactionDate             object
TransactionTime              int64
TransactionAmount (INR)    float64
dtype: object

In [ ]:
recents = transactions.groupby('CustomerID')["TransactionDate"].max().reset_index()
# left join customers table to transactions table
customers = customers.merge(recents,on="CustomerID",how="left")
customers.rename(columns = {"TransactionDate":"most_recent"}, inplace=True)
customers.columns

Index(['CustomerID', 'CustomerDOB', 'CustGender', 'CustLocation',
       'CustAccountBalance', 'most_recent'],
      dtype='object')

In [ ]:
today = datetime.datetime.today().date()
customers["Gap"] = transactions["TransactionDate"].apply(lambda date: (today-date).days)
customers["Num_Transactions"] = transactions.groupby("CustomerID")["TransactionID"].count().values
customers["Sum_Transactions"] = transactions.groupby("CustomerID")["TransactionAmount (INR)"].sum().values

## Scoring RFM columns between 1-5, based on quartiles

In [ ]:
customers["Recency"] = pd.qcut(customers["Gap"],q=5,labels=[5,4,3,2,1]).astype(int)
customers["Frequency"] = customers["Num_Transactions"]
customers["Monetary_Value"] = pd.qcut(customers["Sum_Transactions"],q=5,labels=[1,2,3,4,5]).astype(int)

In [ ]:
customers["RFM_Score"] = customers["Recency"]*100 + customers["Frequency"]*10 + customers["Monetary_Value"]

In [ ]:
customers.head()

CustomerID CustomerDOB CustGender CustLocation  CustAccountBalance  \
0   C5841053     10/1/94          F   JAMSHEDPUR            17819.05   
1   C2142763      4/4/57          M      JHAJJAR             2270.69   
2   C4417068    26/11/96          F       MUMBAI            17874.44   
3   C5342380     14/9/73          F       MUMBAI           866503.21   
4   C9031234     24/3/88          F  NAVI MUMBAI             6714.43   

  most_recent   Gap  Num_Transactions  Sum_Transactions  Recency  Frequency  \
0  2016-02-08  3111                 1            4750.0        1          1   
1  2016-02-08  3111                 2            1455.0        1          2   
2  2016-02-08  3111                 1              30.0        1          1   
3  2016-08-13  3111                 1            5000.0        1          1   
4  2016-02-08  3111                 1             557.0        1          1   

   Monetary_Value  RFM_Score  
0               5        115  
1               4        124  
2               1        111  
3               5        115  
4               3        113

# RFM Scoring

In [ ]:
# the scores range from 111(lowest) to 666(highest), so we can divide in 3 parts
# as 111-259(low), 260-408(med), 409-666(high)

customers["Relationship_Value"] = pd.cut(customers["RFM_Score"], bins=[110, 259, 408, 566], labels=["low", "med", "high"])
customers["Relationship_Value"].head()

0    low
1    low
2    low
3    low
4    low
Name: Relationship_Value, dtype: category
Categories (3, object): ['low' < 'med' < 'high']

In [ ]:
customers.head()

CustomerID CustomerDOB CustGender CustLocation  CustAccountBalance  \
0   C5841053     10/1/94          F   JAMSHEDPUR            17819.05   
1   C2142763      4/4/57          M      JHAJJAR             2270.69   
2   C4417068    26/11/96          F       MUMBAI            17874.44   
3   C5342380     14/9/73          F       MUMBAI           866503.21   
4   C9031234     24/3/88          F  NAVI MUMBAI             6714.43   

  most_recent   Gap  Num_Transactions  Sum_Transactions  Recency  Frequency  \
0  2016-02-08  3111                 1            4750.0        1          1   
1  2016-02-08  3111                 2            1455.0        1          2   
2  2016-02-08  3111                 1              30.0        1          1   
3  2016-08-13  3111                 1            5000.0        1          1   
4  2016-02-08  3111                 1             557.0        1          1   

   Monetary_Value  RFM_Score Relationship_Value  
0               5        115                low  
1               4        124                low  
2               1        111                low  
3               5        115                low  
4               3        113                low

In [ ]:
customers["Relationship_Value"].value_counts()

Relationship_Value
high    221664
low     211232
med     106780
Name: count, dtype: int64

In [ ]:
# Getting the segmentwise customer ID
lows = customers[customers.Relationship_Value=="low"]["CustomerID"]
meds = customers[customers.Relationship_Value=="med"]["CustomerID"]
high = customers[customers.Relationship_Value=="high"]["CustomerID"]
cust_values = {"low":lows,"medium":meds,"high":high}

# Storing

In [ ]:
if not isGenerating:
  customers.to_csv("cust_segmented.csv",index=False)
else:
  extra_cols = ["Loan","Credit_Card","Debit_Card"]
  customers.drop(columns=extra_cols,inplace=True)
  customers.to_csv("fake_cust.csv",index=False)

In [ ]:
customers.isnull().sum()

CustomerID               0
CustomerDOB           1747
CustGender             540
CustLocation            87
CustAccountBalance    1195
most_recent              0
Gap                      0
Num_Transactions         0
Sum_Transactions         0
Recency                  0
Frequency                0
Monetary_Value           0
RFM_Score                0
Relationship_Value       0
dtype: int64

# Showing process

## Original Data

In [ ]:
transactions[["CustomerID","TransactionDate","TransactionAmount (INR)"]].head()

CustomerID TransactionDate  TransactionAmount (INR)
0   C5841053      2016-02-08                     25.0
1   C2142763      2016-02-08                  27999.0
2   C4417068      2016-02-08                    459.0
3   C5342380      2016-02-08                   2060.0
4   C9031234      2016-02-08                   1762.5

## Finding Most Recent Transaction, transaction count, total amount for each customer, from transactions data

In [ ]:
recents = transactions.groupby('CustomerID')["TransactionDate"].max().reset_index()
# left join customers table to transactions table
customers = customers.merge(recents,on="CustomerID",how="left")
customers.rename(columns = {"TransactionDate":"most_recent"}, inplace=True)

In [ ]:
today = datetime.datetime.today().date()
customers["Gap"] = transactions["TransactionDate"].apply(lambda date: (today-date).days)
customers["Num_Transactions"] = transactions.groupby("CustomerID")["TransactionID"].count().values
customers["Sum_Transactions"] = transactions.groupby("CustomerID")["TransactionAmount (INR)"].sum().values

In [ ]:
customers[["CustomerID","most_recent","Num_Transactions","Sum_Transactions"]].head()

CustomerID most_recent most_recent  Num_Transactions  Sum_Transactions
0   C5841053  2016-02-08  2016-02-08                 1            4750.0
1   C2142763  2016-02-08  2016-02-08                 2            1455.0
2   C4417068  2016-02-08  2016-02-08                 1              30.0
3   C5342380  2016-08-13  2016-08-13                 1            5000.0
4   C9031234  2016-02-08  2016-02-08                 1             557.0

## Getting Recency, Frequency, Monetary_Value from those columns, by cutting them in intervals of 1-5, (1-6 for frequency)

In [ ]:
customers["Recency"] = pd.qcut(customers["Gap"],q=5,labels=[5,4,3,2,1]).astype(int)
customers["Frequency"] = customers["Num_Transactions"] # this value is already in range of 1-6
customers["Monetary_Value"] = pd.qcut(customers["Sum_Transactions"],q=5,labels=[1,2,3,4,5]).astype(int)

In [ ]:
customers[["CustomerID","Recency","Frequency","Monetary_Value"]].head()

CustomerID  Recency  Frequency  Monetary_Value
0   C5841053        1          1               5
1   C2142763        1          2               4
2   C4417068        1          1               1
3   C5342380        1          1               5
4   C9031234        1          1               3

## Getting RFM score from individual columns and assigning relationship value based on this score

In [ ]:
customers["RFM_Score"] = customers["Recency"]*100 + customers["Frequency"]*10 + customers["Monetary_Value"]

In [ ]:
# the scores range from 111(lowest) to 666(highest), so we can divide in 3 parts
# as 111-259(low), 260-408(med), 409-666(high)

customers["Relationship_Value"] = pd.cut(customers["RFM_Score"], bins=[110, 259, 408, 666], labels=["low", "med", "high"])

In [ ]:
customers[["CustomerID","RFM_Score","Relationship_Value"]].head()

CustomerID  RFM_Score Relationship_Value
0   C5841053        115                low
1   C2142763        124                low
2   C4417068        111                low
3   C5342380        115                low
4   C9031234        113                low